notes to myself:
* pf
    - Full sequential TRP F C T (F17,T17)
* NCP
    - ...

In [1]:
%matplotlib qt5
import sys
import os
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
from global_sets.component import m

In [2]:
with open('./log/results/opt_PATH_fullseq_pf_F17T16.pickle','rb') as f:
    results_fullseq = pickle.load(f)
with open('./log/results/opt_PATH_grouped_pf_F17T16.pickle','rb') as f:
    results_grouped = pickle.load(f)
with open('./log/results/opt_PATH_simul2_pf_F17T16.pickle','rb') as f:
    results_simul = pickle.load(f)

In [3]:
scenario = {'simul':results_simul,'fullseq':results_fullseq}
scenario = {'simul':results_simul,'fullseq':results_fullseq,'grouped':results_grouped}

In [4]:
tray_flag = [i for i in range(1,21)]
feed_flag = [i for i in range(8,18)]
reactive_flag = [8,9,10,11,12,14,15,16,17,18]
product_flag = [j for j in m.PRODUCT]

In [5]:
obj_data = {key: [] for key in scenario}
T_data = {key: {j:[] for j in feed_flag} for key in scenario}
P_data = {key: {j:[] for j in product_flag} for key in scenario}
P_tray_data = {key: {j:[] for j in product_flag} for key in scenario}
R_data = {key: [] for key in scenario}

F_data = {key: {j:[] for j in feed_flag} for key in scenario}
cat_data = {key: {j:[] for j in feed_flag} for key in scenario}
iter_data = {key: [] for key in scenario}
intervals = {key: [] for key in scenario}

# get path data
for key in scenario:
    for j in feed_flag:
        T_data[key][j] = np.array(scenario[key]['reactive[{}].T'.format(j)]) - 273.15
    for j in product_flag:
        P_data[key][j] = scenario[key]['P_total[{}]'.format(j)]
        P_tray_data[key][j] = scenario[key]['N_tray[{}]'.format(j)]
    R_data[key] = scenario[key]['condenser.L[out]']
    for j in feed_flag:
        F_data[key][j] = scenario[key]['reactive[{}].F'.format(j)]
        cat_data[key][j] = scenario[key]['reactive[{}].cat'.format(j)]
    iter_data[key] = scenario[key]['total_iter']
    intervals[key] = scenario[key]['iter_count']

In [6]:
def get_end_point(total_list,iter_list,data_list):
    tmp_index = []
    tmp_data = []
    for index ,itr in enumerate(iter_list):
        if itr == 2:
            index -= 1
            tmp_index.append(total_list[index])
            tmp_data.append(data_list[index])
    return tmp_index, tmp_data

# Plot style sheet

In [7]:
style.use('seaborn-paper')

In [8]:
fig1, axs = plt.subplots(5,2,figsize=(16,9),sharex=True)

for key in scenario.keys():
    tmp = axs[0][0].plot(iter_data[key],P_data[key]['gasoline'],label=key,alpha=0.8)
    axs[0][0].plot(*get_end_point(iter_data[key],intervals[key],P_data[key]['gasoline']),'D',color=tmp[0].get_color())
    
    tmp = axs[0][1].plot(iter_data[key],P_tray_data[key]['gasoline'],label=key,alpha=0.8)
    axs[0][1].plot(*get_end_point(iter_data[key],intervals[key],P_tray_data[key]['gasoline']),'D',color=tmp[0].get_color())
    
    tmp = axs[1][0].plot(iter_data[key],R_data[key],label=key,alpha=0.8)
    axs[1][0].plot(*get_end_point(iter_data[key],intervals[key],R_data[key]),'D',color=tmp[0].get_color())
    
    tmp = axs[2][0].plot(iter_data[key],P_data[key]['naphtha'],label=key,alpha=0.8)
    axs[2][0].plot(*get_end_point(iter_data[key],intervals[key],P_data[key]['naphtha']),'D',color=tmp[0].get_color())
     
    tmp = axs[3][0].plot(iter_data[key],P_data[key]['diesel'],label=key,alpha=0.8)
    axs[3][0].plot(*get_end_point(iter_data[key],intervals[key],P_data[key]['diesel']),'D',color=tmp[0].get_color())
    
    tmp = axs[3][1].plot(iter_data[key],P_tray_data[key]['diesel'],label=key,alpha=0.8)
    axs[3][1].plot(*get_end_point(iter_data[key],intervals[key],P_tray_data[key]['diesel']),'D',color=tmp[0].get_color())

    tmp = axs[4][0].plot(iter_data[key],P_data[key]['intermediate'],label=key,alpha=0.8)
    axs[4][0].plot(*get_end_point(iter_data[key],intervals[key],P_data[key]['intermediate']),'D',color=tmp[0].get_color())
    
    tmp = axs[4][1].plot(iter_data[key],P_tray_data[key]['intermediate'],label=key,alpha=0.8)
    axs[4][1].plot(*get_end_point(iter_data[key],intervals[key],P_tray_data[key]['intermediate']),'D',color=tmp[0].get_color())
    
for a in axs:
    for x in a:
        x.grid() 
        x.legend()

axs[0][0].set_title('Obj: Gasoline')
axs[1][0].set_title('Reflux Flow')
axs[2][0].set_title('Naphtha')
axs[3][0].set_title('Diesel')
axs[4][0].set_title('Heavy Naphtha')

axs[-1][0].set_xlabel('iter')

plt.tight_layout()
plt.show()

No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [9]:
fig2, axs = plt.subplots(len(feed_flag),3,figsize=(16,9),sharex=True)

for j, stage in enumerate(feed_flag):
    for key in scenario:
        
        tmp = axs[j][0].plot(iter_data[key],T_data[key][stage],label=key,alpha=0.8)
        axs[j][0].plot(*get_end_point(iter_data[key],intervals[key],T_data[key][stage]),'D',color=tmp[0].get_color())
        axs[j][0].set_ylim(190,310)
        
        tmp = axs[j][1].plot(iter_data[key],F_data[key][stage],label=key,alpha=0.8)
        axs[j][1].plot(*get_end_point(iter_data[key],intervals[key],F_data[key][stage]),'D',color=tmp[0].get_color())
        
        tmp = axs[j][2].plot(iter_data[key],cat_data[key][stage],label=key,alpha=0.8)
        axs[j][2].plot(*get_end_point(iter_data[key],intervals[key],cat_data[key][stage]),'D',color=tmp[0].get_color())

for a in axs:
    for x in a:
        x.grid()        

axs[0][0].set_title('Temperature')
axs[0][1].set_title('Feed')
axs[0][2].set_title('Catalyst')

axs[-1][0].set_xlabel('iter')
axs[-1][1].set_xlabel('iter')
axs[-1][2].set_xlabel('iter')

axs[0][0].legend()
axs[0][1].legend()
axs[0][2].legend()

plt.tight_layout()
plt.show()